In [10]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS
from arcpy.sa import *

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Dense_Development.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [3]:
path = os.path.join(THIS_FOLDER, "in_data/SBC_Buildings.shp")
arcpy.CopyFeatures_management(path, "SBC_Buildings")
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
arcpy.Project_management("SBC_Buildings", "SBC_Buildings_rpj", crs)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Dense_Development.gdb\\SBC_Buildings_rpj'>

In [20]:
arcpy.management.MultipartToSinglepart("SBC_Buildings_rpj", "SBC_Buildings_rpj_SP")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Dense_Development.gdb\\SBC_Buildings_rpj_SP'>

In [21]:
arcpy.management.FeatureToPoint("SBC_Buildings_rpj_SP", "SBC_Buildings_Pts")     

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Dense_Development.gdb\\SBC_Buildings_Pts'>

In [35]:
pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(1000, "MAP"))
out_raster = arcpy.ia.Int(pdensOut)
arcpy.RasterToPolygon_conversion(out_raster, "point_density_1000ft", "NO_SIMPLIFY", "VALUE")

pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(2640, "MAP"))
out_raster = arcpy.ia.Int(pdensOut)
arcpy.RasterToPolygon_conversion(out_raster, "point_density_0p5mi", "NO_SIMPLIFY", "VALUE")

pdensOut = PointDensity("SBC_Buildings_Pts", "NONE", 200, NbrCircle(15840, "MAP"))
out_raster = arcpy.ia.Int(pdensOut)
arcpy.RasterToPolygon_conversion(out_raster, "point_density_3mi", "NO_SIMPLIFY", "VALUE")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Dense_Development.gdb\\point_density_3mi'>

In [36]:
arcpy.ClearWorkspaceCache_management()
del pdensOut
del out_raster

In [37]:
dir_path = os.path.join(THIS_FOLDER, "out_data/dense_dev")
shutil.rmtree(dir_path)
os.mkdir(dir_path)


arcpy.conversion.FeatureClassToShapefile("point_density_1000ft", 
                                        os.path.join(THIS_FOLDER, "out_data/dense_dev"))

arcpy.conversion.FeatureClassToShapefile("point_density_0p5mi", 
                                        os.path.join(THIS_FOLDER, "out_data/dense_dev"))

arcpy.conversion.FeatureClassToShapefile("point_density_3mi", 
                                        os.path.join(THIS_FOLDER, "out_data/dense_dev"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\dense_dev'>

In [38]:
field_names = [f.name for f in arcpy.ListFields("point_density_1000ft")]
print(field_names)

['OBJECTID', 'Shape', 'Id', 'gridcode', 'Shape_Length', 'Shape_Area']
